## Implementation Section

In [1]:
import pandas as pd

df_auc = pd.read_csv("BAW_data/df_auction_to2024.csv", sep=";")
df_auc.head()

,date,logistieke location id,crop_type,fpc_index,cropgroupcode_coded,productgroupcode_coded,country of origin iso,sales value id,total_transactions,total_items_sold,total_package,total_revenue,avg_price_per_item
0,2021-05-25,3,BL,fpc_23,19,45,NL,5,218,16860,493.0,10994.70,0.65
1,2021-02-05,3,BL,fpc_27,122,211,NL,5,1557,1847980,12404.0,388688.80,0.21
2,2021-08-02,3,BL,fpc_23,32,67,KE,5,79,47125,656.0,10694.75,0.23
3,2021-05-14,3,BL,fpc_23,41,80,NL,5,236,38680,627.0,18026.90,0.47
4,2021-03-12,1,BL,fpc_24,11,27,NL,5,499,20424,1673.0,17598.28,0.86


In [2]:
df_dir = pd.read_csv("BAW_data/df_direct_to2024.csv", sep=";")
df_dir.head()

,date,logistieke location id,crop_type,fpc_index,cropgroupcode_coded,productgroupcode_coded,country of origin iso,sales value id,total_transactions,total_items_sold,total_package,total_revenue,avg_price_per_item
0,2021-11-10,NaN,PL,fpc_8,319,480,NL,4,432,16463,1291.0,43189.32,2.62
1,2021-03-29,NaN,PL,fpc_7,600,824,NL,4,246,3847,2637.0,63031.20,16.38
2,2022-07-11,NaN,BL,fpc_6,119,193,NL,4,80,108390,991.0,26821.05,0.25
3,2021-01-26,NaN,BL,fpc_13,6,15,NL,4,5066,1192334,13934.0,195380.07,0.16
4,2022-11-16,NaN,PL,fpc_18,380,564,NL,4,231,8004,1042.0,16564.46,2.07


In [3]:
df_fust = pd.read_csv("BAW_data/df_fust_to2024.csv", sep=";")
df_fust.head()

,Date,fustcode,number_received,number_issued
0,2021-01-02,Fc153,0.0,6.0
1,2021-01-02,Fc180,0.0,1.0
2,2021-01-02,Fc206,0.0,150.0
3,2021-01-02,Fc251,0.0,300.0
4,2021-01-02,Fc266,0.0,300.0


In [4]:
df_trol = pd.read_csv("BAW_data/df_trolley_to2024.csv", sep=";")
df_trol.head()

,auctiondate,location,crop_type,fpc_1,fpc_2,fpc_3,fpc_4,fpc_5,fpc_6,fpc_7,...,fpc_20,fpc_21,fpc_22,fpc_23,fpc_24,fpc_25,fpc_26,fpc_27,fpc_28,total_trolleys
0,2021-01-04,Aalsmeer,BL,287,89,94,0,0,126,0,...,0,0,3481,1525,51,112,21,989,0,8964
1,2021-01-04,Aalsmeer,PL,0,0,0,9,550,0,54,...,261,120,0,0,0,0,0,0,22,2010
2,2021-01-04,Naaldwijk,BL,106,55,34,0,0,60,0,...,0,0,942,783,8,56,0,214,0,4150
3,2021-01-04,Naaldwijk,PL,0,0,0,5,444,0,45,...,216,92,0,0,0,0,0,0,9,1523
4,2021-01-04,Rijnsburg,BL,51,59,5,1,60,72,3,...,24,4,570,670,5,5,1,436,0,2491


In [5]:
df_fust_new = (
    df_fust
    .groupby("Date", as_index=False)
    .agg(
        total_issued=("number_issued", "sum"),
        total_returned=("number_received", "sum")
    )
)

In [6]:
df_trol["location"] = df_trol["location"].astype("category")
df_trol["crop_type"] = df_trol["crop_type"].astype("category")

df_trol_new = (
    df_trol
    .drop(columns=["total_trolleys"])
    .melt(
        id_vars=df_trol.columns[:3],
        var_name="fpc_code",
        value_name="trolleys"
    )
)

In [7]:
df_dir = df_dir[df_dir.drop(columns=["logistieke location id"]).notna().all(axis=1)]

In [8]:
df_auc_new = (
    df_auc
    .drop(columns=[
        "cropgroupcode_coded",
        "productgroupcode_coded",
        "country of origin iso",
        "avg_price_per_item"
    ])
    .groupby(
        ["date", "logistieke location id", "crop_type", "fpc_index", "sales value id"],
        as_index=False
    )
    .agg(
        total_transactions=("total_transactions", "sum"),
        total_items_sold=("total_items_sold", "sum"),
        total_package=("total_package", "sum"),
        total_revenue=("total_revenue", "sum")
    )
)

df_auc_new["avg_price_per_item"] = (
    df_auc_new["total_revenue"] / df_auc_new["total_items_sold"]
).round(2)

In [9]:
df_auc_new = (
    df_auc_new
    .melt(
        id_vars=["date", "logistieke location id", "crop_type", "fpc_index", "sales value id"],
        value_vars=[
            "total_transactions",
            "total_items_sold",
            "total_package",
            "total_revenue",
            "avg_price_per_item"
        ],
        var_name="var",
        value_name="value"
    )
    .pivot_table(
        index=["date", "logistieke location id", "crop_type", "fpc_index"],
        columns=["sales value id", "var"],
        values="value"
    )
)

df_auc_new.columns = [
    f"{col[0]}_{col[1]}" for col in df_auc_new.columns
]
df_auc_new = df_auc_new.reset_index()

df_auc_new["logistieke location id"] = df_auc_new["logistieke location id"].replace({
    1: "Aalsmeer",
    2: "Naaldwijk",
    3: "Rijnsburg"
}).astype("category")

numeric_cols_auc = df_auc_new.select_dtypes(include="number").columns
df_auc_new[numeric_cols_auc] = df_auc_new[numeric_cols_auc].fillna(0)
df_auc_new = df_auc_new.dropna()

In [10]:
df_dir_3 = df_dir[df_dir["sales value id"] == 3]
df_dir_4 = df_dir[df_dir["sales value id"] == 4]

In [11]:
df_dir_3_new = (
    df_dir_3
    .drop(columns=[
        "cropgroupcode_coded",
        "productgroupcode_coded",
        "country of origin iso",
        "avg_price_per_item"
    ])
    .groupby(
        ["date", "logistieke location id", "crop_type", "fpc_index", "sales value id"],
        as_index=False
    )
    .agg(
        total_transactions=("total_transactions", "sum"),
        total_items_sold=("total_items_sold", "sum"),
        total_package=("total_package", "sum"),
        total_revenue=("total_revenue", "sum")
    )
)

df_dir_3_new["avg_price_per_item"] = (
    df_dir_3_new["total_revenue"] / df_dir_3_new["total_items_sold"]
).round(2)

In [12]:
df_dir_3_new = (
    df_dir_3_new
    .melt(
        id_vars=["date", "logistieke location id", "crop_type", "fpc_index", "sales value id"],
        value_vars=[
            "total_transactions",
            "total_items_sold",
            "total_package",
            "total_revenue",
            "avg_price_per_item"
        ],
        var_name="var",
        value_name="value"
    )
    .pivot_table(
        index=["date", "logistieke location id", "crop_type", "fpc_index"],
        columns=["sales value id", "var"],
        values="value"
    )
)

df_dir_3_new.columns = [
    f"{col[0]}_{col[1]}" for col in df_dir_3_new.columns
]
df_dir_3_new = df_dir_3_new.reset_index()

In [13]:
df_dir_4_new = (
    df_dir_4
    .drop(columns=[
        "cropgroupcode_coded",
        "productgroupcode_coded",
        "country of origin iso",
        "avg_price_per_item",
        "logistieke location id"
    ])
    .groupby(
        ["date", "crop_type", "fpc_index", "sales value id"],
        as_index=False
    )
    .agg(
        total_transactions=("total_transactions", "sum"),
        total_items_sold=("total_items_sold", "sum"),
        total_package=("total_package", "sum"),
        total_revenue=("total_revenue", "sum")
    )
)

df_dir_4_new["avg_price_per_item"] = (
    df_dir_4_new["total_revenue"] / df_dir_4_new["total_items_sold"]
).round(2)

In [14]:
df_dir_4_new = (
    df_dir_4_new
    .melt(
        id_vars=["date", "crop_type", "fpc_index", "sales value id"],
        value_vars=[
            "total_transactions",
            "total_items_sold",
            "total_package",
            "total_revenue",
            "avg_price_per_item"
        ],
        var_name="var",
        value_name="value"
    )
    .pivot_table(
        index=["date", "crop_type", "fpc_index"],
        columns=["sales value id", "var"],
        values="value"
    )
)

df_dir_4_new.columns = [
    f"{col[0]}_{col[1]}" for col in df_dir_4_new.columns
]
df_dir_4_new = df_dir_4_new.reset_index()

In [15]:
df_dir_new = df_dir_3_new.merge(
    df_dir_4_new,
    on=["date", "crop_type", "fpc_index"],
    how="outer"
)

df_dir_new["logistieke location id"] = df_dir_new["logistieke location id"].replace({
    1: "Aalsmeer",
    2: "Naaldwijk",
    3: "Rijnsburg"
}).astype("category")

numeric_cols_dir = df_dir_new.select_dtypes(include="number").columns
df_dir_new[numeric_cols_dir] = df_dir_new[numeric_cols_dir].fillna(0)

df_dir_new = df_dir_new.dropna()

In [16]:
new1 = df_dir_new.merge(
    df_auc_new,
    on=["date", "crop_type", "fpc_index", "logistieke location id"],
    how="outer"
)

numeric_cols_new1 = new1.select_dtypes(include="number").columns
new1[numeric_cols_new1] = new1[numeric_cols_new1].fillna(0)

new1 = new1.dropna()

In [17]:
new2 = new1.merge(
    df_trol_new,
    left_on=["date", "crop_type", "fpc_index", "logistieke location id"],
    right_on=["auctiondate", "crop_type", "fpc_code", "location"],
    how="inner"
)

In [18]:
merged_df = new2.merge(
    df_fust_new,
    left_on="date",
    right_on="Date",
    how="left"
)

In [19]:
merged_df = merged_df.drop(columns=["Date", "auctiondate", "location", "fpc_code"])
merged_df = merged_df.dropna()
merged_df.head()

,date,logistieke location id,crop_type,fpc_index,3_avg_price_per_item,3_total_items_sold,3_total_package,3_total_revenue,3_total_transactions,4_avg_price_per_item,...,1_total_revenue,1_total_transactions,5_avg_price_per_item,5_total_items_sold,5_total_package,5_total_revenue,5_total_transactions,trolleys,total_issued,total_returned
0,2021-01-04,Aalsmeer,BL,fpc_1,0.21,17510.0,273.0,3683.38,48.0,0.26,...,18960.20,717.0,0.24,312850.0,4770.0,76058.00,1237.0,287,621375.0,771606.0
1,2021-01-04,Naaldwijk,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,5483.50,227.0,0.26,110970.0,1762.0,29005.80,553.0,106,621375.0,771606.0
2,2021-01-04,Rijnsburg,BL,fpc_1,0.00,0.0,0.0,0.00,0.0,0.00,...,4349.40,167.0,0.26,50810.0,786.0,13353.90,302.0,51,621375.0,771606.0
3,2021-01-04,Aalsmeer,BL,fpc_10,0.27,4250.0,59.0,1146.50,13.0,0.27,...,18427.65,674.0,0.20,356840.0,4085.0,71239.65,1114.0,191,621375.0,771606.0
4,2021-01-04,Naaldwijk,BL,fpc_10,0.00,0.0,0.0,0.00,0.0,0.00,...,7674.80,292.0,0.19,239939.0,2952.0,45257.18,918.0,138,621375.0,771606.0


In [20]:
merged_df.shape[0]

72080